In [4]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
import json

In [5]:
def parse(path):
  g = gzip.open("review-New_York_10.json.gz", 'r')
  for l in g:
    yield json.loads(l)

In [6]:
dataset = []
for l in parse("review-New_York_10.json.gz"):
  dataset.append(l)